In [ ]:
import os 
from pathlib import Path

import pandas as pd
import numpy as np
import anndata as ad

In [ ]:
df_obs = ad.read_h5ad("/home/x-aklein2/projects/aklein/BICAN/BG/data/BICAN_BG_CPSAM_annotated_v2.h5ad").obs.copy()
df_obs.head()

In [ ]:
header = """#!/bin/bash
# FILENAME = rethresh.sh 

#SBATCH -A mcb130189
#SBATCH -J rethresh_cp2_{EXP}
#SBATCH -p shared
#SBATCH --time=2:30:00
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --mem=64gb
#SBATCH -o /home/x-aklein2/projects/aklein/BICAN/BG/logs/rethresh_cp2_{EXP}.out
#SBATCH -e /home/x-aklein2/projects/aklein/BICAN/BG/logs/rethresh_cp2_{EXP}.out
#SBATCH --export=ALL

module load modtree/cpu
module list

export PATH="/home/x-aklein2/.pixi/bin:$PATH"
cd /anvil/projects/x-mcb130189/aklein/SPIDA
"""

In [ ]:
template = """
echo -e "\\n Processing {EXP} - {REG} -- COUNTER {i} / {LEN} \\n"

# Loading in the segmentation into the zarr store
pixi run -e preprocessing \\
    python -m spida.S load_segmentation_region \\
    {EXP} \\
    {REG} \\
    /home/x-aklein2/projects/aklein/BICAN/data/segmented/{EXP}/cellpose \\
    --type vpt \\
    --prefix_name CPS \\
    --plot

# FILTERING 
pixi run -e preprocessing \\
    python -m spida.P filter_cells_region \\
    {EXP} \\
    {REG} \\
    CPS \\
    --plot \\
    --cutoffs_path /home/x-aklein2/projects/aklein/BICAN/data/filtering_cutoffs_cp2.json

# SETUP ADATA 
pixi run -e preprocessing \\
    python -m spida.P setup_adata_region \\
    {EXP} \\
    {REG} \\
    CPS \\
    --suffix filt \\
    --plot

"""

In [ ]:
for _brain_region in df_obs['brain_region'].unique():
    df_sub = df_obs[df_obs['brain_region'] == _brain_region]
    fout = header.format(EXP=_brain_region)
    cc = 1
    for _experiment in df_sub['experiment'].unique():
        for _region in df_sub[df_sub['experiment'] == _experiment]['region'].unique():
            fout += template.format(EXP=_experiment, REG=_region, i=cc, LEN=8)
            cc += 1
    script_path = Path(f"/home/x-aklein2/projects/aklein/BICAN/BG/image_processing/_cp_thresh/rethresh_cp2_{_brain_region}.sh")
    with open(script_path, "w") as f:
        f.write(fout)
    os.system(f"sbatch {script_path}")